In [1]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

import os
import sys
import cv2
import math
import numpy as np
import torch

from models.experimental import attempt_load
from utils.torch_utils import select_device
from utils.general import check_img_size, non_max_suppression, xywh2xyxy, xyxy2xywh

import matplotlib.pyplot as plt

MODEL_PATH  = 'weights/yolov5s_thalas.pt'
IMG_SIZE    = 640
SAVE_DIR    = 'output'


In [2]:
im = cv2.imread('2.png')


In [ ]:
plt.imshow(im)

In [ ]:
def output_to_target(output):
    # Convert model output to target format [batch_id, class_id, x, y, w, h, conf]
    targets = []
    for i, o in enumerate(output):
        for *box, conf, cls in o.cpu().numpy():
            targets.append([i, cls, *list(*xyxy2xywh(np.array(box)[None])), conf])
    return np.array(targets)

def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
#     color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))

    print('img: ', img.shape)

    cv2.rectangle(img, c1, c2, color, tl)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)


def plot_image(image, target, output_dir, names):
    if isinstance(image, torch.Tensor):
        image = image.cpu().float().numpy()
    if isinstance(target, torch.Tensor):
        target = target.cpu().numpy()
    # un-normalise
    if np.max(image[0]) <= 1:
        image *= 255
    tl = 3  # line thickness
    tf = max(tl - 1, 1)  # font thickness
    bs, _, h, w = image.shape
     # Check if we should resize
    scale_factor = IMG_SIZE / max(h, w)
    image = image[0].transpose(1, 2, 0)
    
    if scale_factor < 1:
        h = math.ceil(scale_factor * h)
        w = math.ceil(scale_factor * w)
        image = cv2.resize(image, (w, h))
    if len(target) > 0:
        boxes = xywh2xyxy(target[:, 2:6]).T
        classes = target[:, 1]
        labels = target.shape[1] == 6  # labels if no conf column
        conf = None if labels else target[:, 6]  # check for confidence presence (label vs pred)
        if boxes.shape[1]:
            if boxes.max() <= 1.01:  # if normalized with tolerance 0.01
                boxes[[0, 2]] *= w  # scale to pixels
                boxes[[1, 3]] *= h
            elif scale_factor < 1:  # absolute coords need scale if image scales
                boxes *= scale_factor
        for j, box in enumerate(boxes.T):
            cls = int(classes[j])
            cls = names[cls] if names else cls
            if labels or conf[j] > 0.25:  # 0.25 conf thresh
                label = '%s' % cls if labels else '%s %.1f' % (cls, conf[j])
                plot_one_box(box, image, label=label, color=(255, 0, 0), line_thickness=tl)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(str(output_dir), image)

In [ ]:
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    
    return img, ratio, (dw, dh)


def preprocess(image, img_size = IMG_SIZE, stride = 32, pad = 0.5):
    h0, w0 = image.shape[:2]  # orig hw    
    r = IMG_SIZE / max(h0, w0)  # resize image to img_size
    if r != 1:  # always resize down, only resize up if training with augmentation
        interp = cv2.INTER_AREA if r < 1 else cv2.INTER_LINEAR
        image = cv2.resize(image, (int(w0 * r), int(h0 * r)), interpolation=interp)
    
    ar = w0/h0
    if ar < 1:
        shapes = [ar, 1]
    elif ar > 1:
        shapes = [1, 1 / ar]
        
    img_shape = np.ceil(np.array(shapes) * img_size / stride + pad).astype(np.int) * stride

    img = letterbox(image, img_shape, stride=stride)[0]
    
    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    return img

def inference(image, name):
    # Select device
    device = select_device(batch_size=1)
    
    # Load model
    model = attempt_load(MODEL_PATH, map_location=device)
    gs = max(int(model.stride.max()), 32)  # grid size (max stride)
    imgsz = check_img_size(IMG_SIZE, s=gs)  # check img_size
    # Half
    half = device.type != 'cpu'  # half precision only supported on CUDA
    if half:
        model.half()  
    
    # Configure
    model.eval()
    
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    
    classes = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}
    
    # preprocess image
    image = preprocess(image)
    image = torch.from_numpy(np.expand_dims(image, axis=0))
    image = image.to(device, non_blocking=True)
    image = image.half() if half else image.float()  # uint8 to fp16/32
    image /= 255.0  # 0 - 255 to 0.0 - 1.0
#     nb, _, height, width = img.shape  # batch size, channels, height, width

    with torch.no_grad():
        # Run model
        out, train_out = model(image, augment=False)  # inference and training outputs
        out = non_max_suppression(out, conf_thres=0.001, iou_thres=0.5, multi_label=True)
        
        
    targets = output_to_target(out)
    
    # draw boxes
    plot_image(image, targets, name, classes)
    
    print("************ DONE **************")

In [ ]:
img_path ='2.png'
image = cv2.imread(img_path)

In [ ]:
img_path ='2.png'
image = cv2.imread(img_path)

# Select device
device = select_device(batch_size=1)

# Load model
model = attempt_load(MODEL_PATH, map_location=device)
gs = max(int(model.stride.max()), 32)  # grid size (max stride)
imgsz = check_img_size(IMG_SIZE, s=gs)  # check img_size
# Half
half = device.type != 'cpu'  # half precision only supported on CUDA
if half:
    model.half()  

# Configure
model.eval()

if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once

classes = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}

# preprocess image
image = preprocess(image)
image = torch.from_numpy(np.expand_dims(image, axis=0))
image = image.to(device, non_blocking=True)
image = image.half() if half else image.float()  # uint8 to fp16/32
image /= 255.0  # 0 - 255 to 0.0 - 1.0
#     nb, _, height, width = img.shape  # batch size, channels, height, width

with torch.no_grad():
    # Run model
    out, train_out = model(image, augment=False)  # inference and training outputs
    out = non_max_suppression(out, conf_thres=0.001, iou_thres=0.5, multi_label=True)


target = output_to_target(out)



In [ ]:
if isinstance(image, torch.Tensor):
    image = image.cpu().float().numpy()
if isinstance(target, torch.Tensor):
    target = target.cpu().numpy()
# un-normalise
if np.max(image[0]) <= 1:
    image *= 255

In [ ]:
im = cv2.imread('2.png')
box = np.array([10,10 , 200, 200])
plot_one_box(box, im, label='label', color=(255, 0, 0), line_thickness=1)

In [ ]:
im.shape, type(im), np.unique(im)

In [ ]:
# plt.imshow(im)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import cv2
im2 = cv2.imread('2.png')

In [ ]:
plt.imshow(im2)

In [ ]:
im2.shape, type(im2), np.unique(im2)